In [18]:
#example based on https://keras.io/guides/working_with_rnns/
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [19]:
# Load data
#dataset link https://www.kaggle.com/datasets/fhamborg/news-articles-sentiment
data_train = pd.read_json("train.jsonl", lines=True)
data_test = pd.read_json("test.jsonl", lines=True)
#data_train.head()
data_test.head()

x_train = data_train.sentence
y_train = data_train.polarity

x_test = data_test.sentence
y_test = data_test.polarity

In [44]:
print(len(data_train), "Training")
print(len(data_test), "Test")
#data_train = data_train.drop_duplicates(subset='sentence', keep='first')
#data_test = data_test.drop_duplicates(subset='sentence', keep='first')

7739 Training
747 Test


In [45]:
# Preparing sentence and label for training
X_train = data_train['sentence']
y_train = data_train['polarity']

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=max_length)

y_train_categorical = to_categorical(y_train, num_classes=3)

In [46]:
v_size = len(tokenizer.word_index) + 1
print(v_size)

19585


In [47]:
# Testing data prep
X_val = data_test['sentence']
y_val = data_test['polarity']

X_val = tokenizer.texts_to_sequences(X_val)
X_val = pad_sequences(X_val, maxlen=max_len)

# Convert labels to categorical (if not already in one-hot encoding)
y_val_categorical = to_categorical(y_val, num_classes=3)

In [48]:
#check dataset for all polarity options to see if there is an unbalanced level of training data

df = pd.read_json('train.jsonl', lines=True)
rows_with_precision_minus_one = df[df['polarity'] == -1]
# Get the count of rows with precision -1
count_precision_minus_one = len(rows_with_precision_minus_one)
print(f"Number of rows with polarity -1: {count_precision_minus_one}")

rows_with_precision_minus_one = df[df['polarity'] == 0]
# Get the count of rows with precision 0
count_precision_minus_one = len(rows_with_precision_minus_one)
print(f"Number of rows with polarity 0: {count_precision_minus_one}")

rows_with_precision_minus_one = df[df['polarity'] == 1]
# Get the count of rows with precision 1
count_precision_minus_one = len(rows_with_precision_minus_one)
print(f"Number of rows with polarity 1: {count_precision_minus_one}")

Number of rows with polarity -1: 3316
Number of rows with polarity 0: 3028
Number of rows with polarity 1: 2395


Slight bias will resamlpe the dataset os they have more similar spread of polarity


In [50]:
#resample data, oversampling
from sklearn.utils import resample
import pandas as pd

# Assuming your DataFrame is named 'df'
# Replace 'your_dataset.csv' with the actual file path or name
df = pd.read_json('train.jsonl', lines=True)

# Separate classes
df_class_minus_one = df[df['polarity'] == -1]
df_class_0 = df[df['polarity'] == 0]
df_class_1 = df[df['polarity'] == 1]

# Oversample minority classes (class -1 and class 1 in this case)
df_class_minus_one_oversampled = resample(df_class_minus_one, replace=True, n_samples=len(df_class_0), random_state=42)
df_class_1_oversampled = resample(df_class_1, replace=True, n_samples=len(df_class_0), random_state=42)

# Combine oversampled minority classes with majority class
df_oversampled = pd.concat([df_class_minus_one_oversampled, df_class_0, df_class_1_oversampled])

# Shuffle the dataframe to mix the classes
df_oversampled = df_oversampled.sample(frac=1, random_state=42)

# Display the counts after oversampling
print(df_oversampled['polarity'].value_counts())


polarity
 1    3028
 0    3028
-1    3028
Name: count, dtype: int64


In [51]:
#now retrain a new model and tokenise and split the data
max_length = 200

x_predict = df_oversampled['sentence']
y_label = df_oversampled['polarity']

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_predict)
x_predict = tokenizer.texts_to_sequences(x_predict)
x_predict = pad_sequences(x_predict, maxlen=max_length)

In [52]:
#splitting over sampled data into a training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
x_predict, x_p, y_label, y_l = train_test_split(x_predict, y_label, test_size=0.2, random_state=42)

y_label_categorical = to_categorical(y_label, num_classes=3)
y_l_categorical = to_categorical(y_l, num_classes=3)

# Print the lengths of the training and validation sets
print(len(x_predict), "Training sequences")
print(len(x_p), "Validation sequences")

print(len(y_label), "Training sequences")
print(len(y_l), "Validation sequences")

7267 Training sequences
1817 Validation sequences
7267 Training sequences
1817 Validation sequences


In [53]:
#creating new model

embedding_dim = 50
vocab_size = 18000
max_length = 200

m1 = Sequential()
m1.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
m1.add(LSTM(units=150, dropout=0.3, recurrent_dropout=0.3))
m1.add(Dense(units=3, activation='softmax'))

m1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
m1.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 200, 50)           900000    
                                                                 
 lstm_5 (LSTM)               (None, 150)               120600    
                                                                 
 dense_5 (Dense)             (None, 3)                 453       
                                                                 
Total params: 1021053 (3.90 MB)
Trainable params: 1021053 (3.90 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
# Train the model
m1.fit(x_predict, y_label_categorical, epochs=10, batch_size=64, validation_data=(x_p, y_l_categorical))

# Evaluate the model
loss, accuracy = m1.evaluate(x_p, y_l_categorical)
print(f'Loss: {loss}, Accuracy: {accuracy * 100:.2f}%')

Epoch 1/10
114/114 [==============================] - 32s 265ms/step - loss: 1.0316 - accuracy: 0.4569 - val_loss: 0.9177 - val_accuracy: 0.5482
Epoch 2/10
114/114 [==============================] - 33s 287ms/step - loss: 0.7044 - accuracy: 0.6885 - val_loss: 0.7681 - val_accuracy: 0.6643
Epoch 3/10
114/114 [==============================] - 34s 295ms/step - loss: 0.4459 - accuracy: 0.8353 - val_loss: 0.7492 - val_accuracy: 0.7056
Epoch 4/10
114/114 [==============================] - 33s 292ms/step - loss: 0.2805 - accuracy: 0.9027 - val_loss: 0.7614 - val_accuracy: 0.7166
Epoch 5/10
114/114 [==============================] - 33s 293ms/step - loss: 0.1991 - accuracy: 0.9335 - val_loss: 0.9090 - val_accuracy: 0.7100
Epoch 6/10
114/114 [==============================] - 34s 296ms/step - loss: 0.1557 - accuracy: 0.9513 - val_loss: 0.8410 - val_accuracy: 0.7325
Epoch 7/10
114/114 [==============================] - 47s 416ms/step - loss: 0.1380 - accuracy: 0.9542 - val_loss: 0.8876 - val_ac